# DNBLab Jupyter Notebook Tutorial 

## Titelanalyse Teil 2

In diesem Tutorial-Teil nutzen wir nun die vorher geschriebene Funktion, um bestimmte bibliographischen Informationen aus der MARC21-XML-Datei in ein Pandas Dataframe, oder alternativ eine Excel-Tabelle, zu überführen. Da wir später auch noch andere Fragestellungen mit Hilfe von Python an die Daten stellen möchten, legen wir das Dataframe zunächst für alle in der XML-Datei enthaltenen bibliographischen Einträge an.   

Zunächst importieren wir hierfür einen Teil unseres Codes aus dem vorherigen Tutorial: 

In [2]:
# Laden benötigter Python-Bibliotheken:
import xml.etree.ElementTree as ET
import pandas as pd

# Laden der MARC21-XML-Datei:
tree = ET.parse('data/dataset_tutorial_eco.xml')  # Laden der MARC-Datei in ElementTree 
root = tree.getroot()                             # Laden des Root-Verzeichnisses des XML

records = list(root)
total_records = len(records)  


# Ausgabe der Gesamtzahl der in der Datei vorhandenen Records:
print("Gesamtzahl bibliographischer Einträge: ",total_records)


# Funktion get_subfield_text:
def get_subfield_text(element, tag, subfield):
    if element.attrib['tag'] == tag:
            for subelement in element:
                if subelement.attrib['code'] == subfield:
                    return subelement.text
    return None
    

Gesamtzahl bibliographischer Einträge:  2828


Im folgenden überlegen wir, welche bibliographischen Angaben wir der Datei entnehmen wollen. Um im Sinne des Auffindes von Literatur alle wichtigen Angaben zu erhalten, brauchen wir Angaben zur Urheberschaft (Autor\*in etc.), Titel, Verlag, Verlagsort und Erscheinungsdatum. Wir arbeiten zwar bereits mit Daten, die sich auf Publikationen aus der Sachgruppe "Wirtschaft" beschränken, lassen uns zu Kontroll- und Übungszwecken aber auch diese mit ausgeben. Auch die DNB-eigene Identifiktationsnummer des Datensatzes werden wir zur besseren Überprüfbarkeit der Ergebnisse extrahieren. 

Die Liste der von uns gesuchten Daten sieht also folgendermaßen aus: 

* ID der DNB
* DNB-Sachgruppe
* Autor*in bzw. Herausgeber*in / Creator
* Titel / Title
* Verlag / Publisher
* Verlagsort / Place of publication
* Datum der Veröffentlichung / Date of publication


Um die Daten zu erhalten, schreiben wir uns eine Routine, die diese Angaben aus der XML-Datei holt und in ein Dataframe ablegt. Dafür legen wir die verschiedenen Elemente zunächst in für diese vorgesehene Listen ab und führen die Listen danach zu einem Dataframe zusammen. Die verschiedenen Angaben bzw. Elemente fragen wir dabei in der Reihenfolge ab, in der sie auch in der XML-Datei liegen: 


In [4]:
#Anlegen der zu befüllenden Listen: 
idcoll = []
ddccoll = []
authorcoll = []
titlecoll = []
placecoll = []
publishercoll = []
pubyear = []
interimlist = []     #Diese Liste wird nur zum Zwischenspeichern verwendet


for record in records:  
        
    for child in record.findall("{http://www.loc.gov/MARC21/slim}controlfield[@tag='001']"):   
        ident = child.text
        idcoll.append(ident)
       
    #Suche und Übernahme der Sachgruppe/DDC: 
    for child in record.findall("{http://www.loc.gov/MARC21/slim}datafield[@tag='082']"):
        for subelement in child:
            if subelement.attrib['code'] == "a":
                interimlist = []
                ddc = subelement.text
                interimlist.append(ddc)
            else:
                interimlist.append("Nicht vorhanden!")
                
    #Erstelle Liste mit jeweils nur dem ersten Element aus der jeweils vorangegangenen Interimsliste der Sachgruppen: 
    ddccoll.append(interimlist[0]) 
    
    
    #Suche und Übernahme von Autor*innen bzw. Herausgeber*innen: 
    for child in record.findall("{http://www.loc.gov/MARC21/slim}datafield[@tag='100']"):
        interimlist2 = []
        for subelement in child:
            if subelement.attrib['code'] == "a":
                mainauthor = subelement.text
                interimlist2.append(mainauthor)                
            elif not child in record.findall("{http://www.loc.gov/MARC21/slim}datafield[@tag='100']"):  
                for child in record.find("{http://www.loc.gov/MARC21/slim}datafield[@tag='700']"): #finde erstes Kindelement "700"
                    for subelement in child:
                        if subelement.attrib['code'] == "a":
                            alternate = subelement.text
                            interimlist2.append(alternate)
                        else:
                            interimlist2.append("missing")
             
    authorcoll.append(interimlist2[0])
  
    #Suche und Übernahme des Titels: 
    for child in record.findall("{http://www.loc.gov/MARC21/slim}datafield"):             
        title = get_subfield_text(child, "245", "a") 
        if title:
            titlecoll.append(title)
        
            
    
    #Suche und Übernahme des Publikationsjahres: 
    for child in record.findall("{http://www.loc.gov/MARC21/slim}datafield[@tag='264']"):   
        interimlist3 = []
        for subelement in child:
            if subelement.attrib['code'] == "c":
                rdapub = subelement.text
                interimlist3.append(rdapub)                
            elif not child in record.findall("{http://www.loc.gov/MARC21/slim}datafield[@tag='264']"):  
                for child in record.find("{http://www.loc.gov/MARC21/slim}datafield[@tag='260']"): #finde erstes Kindelement "260"
                    for subelement in child:
                        if subelement.attrib['code'] == "c":
                            oldpub = subelement.text
                            interimlist3.append(oldpub)
                        else:
                            interimlist3.append("Nicht vorhanden")
             
    pubyear.append(interimlist3[0])

    
    #Suche und Übernahme des Publikationsortes: 
    for child in record.findall("{http://www.loc.gov/MARC21/slim}datafield[@tag='264']"):
        for subelement in child:
            if subelement.attrib['code'] == "a":
                interimlist = []
                place = subelement.text
                interimlist.append(place)
            else:
                interimlist.append("Nicht vorhanden!")
                 
    placecoll.append(interimlist[0]) 
    
    
    #Suche und Übernahme des Verlages: 
    for child in record.findall("{http://www.loc.gov/MARC21/slim}datafield[@tag='264']"):
        for subelement in child:
            if subelement.attrib['code'] == "b":
                interimlist = []
                publisher = subelement.text
                interimlist.append(publisher)
            else:
                interimlist.append("Nicht vorhanden!")
                
    publishercoll.append(interimlist[0]) 

    
#Anzeige der Anzahl der verschiedenen Elemente in den jeweiligen Listen:             
print("Elemente in ID-Collection:", len(idcoll))    
print("Elemente in DDC-Collection:", len(ddccoll))  
print("Elemente in Author-Collection:", len(authorcoll))
print("Elemente in Titel-Collection:", len(titlecoll))
print("Elemente in Publikationsjahr-Collection:", len(pubyear))
print("Elemente in Ort-Collection:", len(placecoll))
print("Elemente in Verlag-Collection:", len(publishercoll))
              

Elemente in ID-Collection: 2828
Elemente in DDC-Collection: 2828
Elemente in Author-Collection: 2828
Elemente in Titel-Collection: 2828
Elemente in Publikationsjahr-Collection: 2828
Elemente in Ort-Collection: 2828
Elemente in Verlag-Collection: 2828


Wir können hier überprüfen, dass wir in jeder Unterliste die selbe Anzahl an Einträgen vorfinden - diese sollte auch mit der Gesamtzahl an gefundenen bibliographischen Einträgen übereinstimmen. Ist dies der Fall, können wir die Listen im nächsten Schritt zu einer Tabelle zusammenführen und anschließend in ein Dataframe überführen: 


In [5]:
#Zusammenführen der Ergebnisse in eine Tabelle: 
result = {'Identifikator' : idcoll, 'Autor*in' : authorcoll, 'Titel' : titlecoll, 'Ort' : placecoll, 'Verlag' : publishercoll, 'Jahr' : pubyear, 'Sachgruppe/DDC' : ddccoll}

#Erstellen eines Dataframes aus den Ergebnissen: 
df = pd.DataFrame(result)
#print(result)

#Gibt die ersten 5 Zeilen des Dataframes aus (zur Kontrolle):
df.head() 

#df.to_csv("AlleTitel.csv", index=False) 


,Identifikator,Autor*in,Titel,Ort,Verlag,Jahr,Sachgruppe/DDC
0,1043718966,"Juarez Hernandez, Maria Isabel",A cross-cultural study of motivational factors...,Göttingen,Niedersächsische Staats- und Universitätsbib...,2010,650
1,1019902175,"Blaskowitz, Oliver Jim",A forecast evaluation of PCA-based adaptive ...,Kiel,Universitätsbibliothek Kiel,2010,330
2,1012057232,"March, Christoph",A new perspective on social learning,330,330,2010,330
3,1009567691,"Nubbemeyer, Elmar",A reconsideration of full-cost pricing,338.521,338.521,2010,338.521
4,1045290823,"Plambeck, Hauke Heinrich Friedrich",A study on the impact of mobile telecommunicat...,Frankfurt am Main,Univ.-Bibliothek Frankfurt am Main,2010,330


Wir können uns auch die Gesamtlänge des Dataframes folgendermaßen anzeigen lassen: 

In [6]:
rows_dataframe = df[df.columns[0]].count()
print("Gesamtzahl an Einträgen:", rows_dataframe)

Gesamtzahl an Einträgen: 2828


Da wir uns vorher bereits die Anzahl der jeweiligen Elemente pro Variable haben anzeigen lassen, ist das Ergebnis in diesem Fall wenig überraschend und dient vor allem dem Abgleich.

Mit diesem Dataframe können wir nun auf verschiedenste Arten weiterarbeiten - weitere Tutorials folgen! 